# Análise Exploratória de dados de eleições

## Objetivo

Analisar um conjunto de informações sobre candidatos e pleitos eleitorais, de modo a gerar um modelo razoável de características adequadas a obter um cargo eletivo

## Preparação dos dados

Importações

In [13]:
import pandas as pd
import numpy as np
from sklearn import decomposition
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

import matplotlib.pyplot as plt

Leitura dos dados de candidato

In [2]:
tabela_candidato = pd.read_csv('datasets/candidatos.csv', dtype = {
    "ano_eleicao": int, 
    "codigo_eleicao": float,                            #NAN
    "codigo_tipo_eleicao": float,                         #NAN
    "data_eleicao": object, 
    "descricao_eleicao": object, 
    "nome_tipo_eleicao": object, 
    "tipo_abrangencia_eleicao": object, 
    "codigo_totalizacao_turno": float,                    #NAN
    "descricao_totalizacao_turno": object, 
    "numero_turno": int, 
    "descricao_ue": object, 
    "sigla_ue": object, 
    "sigla_uf": object, 
    "codigo_cor_raca": float,                             #NAN
    "codigo_estado_civil": int, 
    "codigo_genero": int, 
    "codigo_grau_instrucao": int, 
    "codigo_municipio_nascimento": int, 
    "codigo_nacionalidade": int, 
    "codigo_ocupacao": int, 
    "cpf": int, 
    "data_nascimento": object, 
    "descricao_cor_raca": object, 
    "descricao_estado_civil": object, 
    "descricao_genero": object, 
    "descricao_grau_instrucao": object, 
    "descricao_nacionalidade": object, 
    "descricao_ocupacao": object, 
    "email": object, 
    "nome": object, 
    "nome_municipio_nascimento": object, 
    "nome_social": object, 
    "sigla_uf_nascimento": object, 
    "titulo_eleitoral": float,                            #NAN
    "codigo_legenda": float,                              #NAN
    "composicao_legenda": object, 
    "nome_legenda": object, 
    "nome_partido": object, 
    "numero_partido": int, 
    "sigla_legenda": object, 
    "sigla_partido": object, 
    "tipo_agremiacao": object, 
    "codigo_cargo": int, 
    "codigo_detalhe_situacao_candidatura": float,         #NAN
    "codigo_situacao_candidatura": int, 
    "concorre_reeleicao": object, 
    "declara_bens": object, 
    "descricao_cargo": object, 
    "descricao_detalhe_situacao_candidatura": object, 
    "descricao_situacao_candidatura": object, 
    "despesa_maxima_campanha": float, 
    "idade_data_eleicao": float,                          #NAN
    "idade_data_posse": float,                            #NAN
    "nome_urna": object, 
    "numero_processo_candidatura": object,
    "numero_protocolo_candidatura": float,                #NAN
    "numero_sequencial": int, 
    "numero_urna": float,                                 #NAN 
    "pergunta": object})
tabela_candidato.head()

,ano_eleicao,codigo_eleicao,codigo_tipo_eleicao,data_eleicao,descricao_eleicao,nome_tipo_eleicao,tipo_abrangencia_eleicao,codigo_totalizacao_turno,descricao_totalizacao_turno,numero_turno,...,descricao_situacao_candidatura,despesa_maxima_campanha,idade_data_eleicao,idade_data_posse,nome_urna,numero_processo_candidatura,numero_protocolo_candidatura,numero_sequencial,numero_urna,pergunta
0,1996,NaN,NaN,NaN,ELEICOES 1996,NaN,NaN,4.0,NAO ELEITO,1,...,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,-1,11.0,NaN
1,1996,NaN,NaN,NaN,ELEICOES 1996,NaN,NaN,4.0,NAO ELEITO,1,...,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,-1,13.0,NaN
2,1996,NaN,NaN,NaN,ELEICOES 1996,NaN,NaN,1.0,ELEITO,1,...,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,-1,15.0,NaN
3,1996,NaN,NaN,NaN,ELEICOES 1996,NaN,NaN,NaN,NaN,1,...,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,-1,111.0,NaN
4,1996,NaN,NaN,NaN,ELEICOES 1996,NaN,NaN,NaN,NaN,1,...,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,-1,151.0,NaN


Listagem dos pleitos presentes retorna 254 eleições, dentre 8 eleições ordinárias, 104 suplementares e 142 não informadas

In [3]:
df_pleitos = tabela_candidato\
    .fillna(-9999)\
    .groupby([
        'codigo_eleicao', 'ano_eleicao', 'data_eleicao', 
        'codigo_tipo_eleicao', 'nome_tipo_eleicao', 'descricao_eleicao'
    ])\
    .size()\
    .reset_index()\
    .rename(columns={0:'num_candidatos'})\
    .sort_values(by=['codigo_tipo_eleicao'])

df_pleitos\
    .groupby(['codigo_tipo_eleicao', 'nome_tipo_eleicao'])\
    .size()\
    .reset_index()\
    .rename(columns={0:'count'})\
    .sort_values(by=['codigo_tipo_eleicao'])

,codigo_tipo_eleicao,nome_tipo_eleicao,count
0,-9999.0,-9999,142
1,1.0,ELEICAO SUPLEMENTAR,104
2,2.0,ELEICAO ORDINARIA,8


## Seleção dos dados (features e linhas)

Considerando a quantidade de dados razoável e a relativa abundância de informações, optou-se por dar foco apenas nas eleições ordinárias.

In [4]:
tabela_candidato\
    .loc[tabela_candidato['codigo_tipo_eleicao'] == 2.0]\
    .fillna(-9999)\
    .groupby([
        'codigo_eleicao', 'ano_eleicao', 'data_eleicao', 'descricao_eleicao', 
        'nome_tipo_eleicao', 'tipo_abrangencia_eleicao'
    ])\
    .size()\
    .reset_index()\
    .rename(columns={0:'count'})

,codigo_eleicao,ano_eleicao,data_eleicao,descricao_eleicao,nome_tipo_eleicao,tipo_abrangencia_eleicao,count
0,143.0,2014,05/10/2014,ELEICOES GERAIS 2014,ELEICAO ORDINARIA,FEDERAL,26162
1,144.0,2014,26/10/2014,ELEICOES GERAIS 2014,ELEICAO ORDINARIA,FEDERAL,60
2,220.0,2016,02/10/2016,ELEICOES MUNICIPAIS 2016,ELEICAO ORDINARIA,MUNICIPAL,496895
3,221.0,2016,30/10/2016,ELEICOES MUNICIPAIS 2016,ELEICAO ORDINARIA,MUNICIPAL,228
4,295.0,2018,07/10/2018,ELEICAO GERAL FEDERAL 2018,ELEICAO ORDINARIA,FEDERAL,28
5,296.0,2018,28/10/2018,ELEICAO GERAL FEDERAL 2018,ELEICAO ORDINARIA,FEDERAL,4
6,297.0,2018,07/10/2018,ELEICOES GERAIS ESTADUAIS 2018,ELEICAO ORDINARIA,ESTADUAL,29057
7,298.0,2018,28/10/2018,ELEICOES GERAIS ESTADUAIS 2018,ELEICAO ORDINARIA,ESTADUAL,56


Além disso, foram removidas as colunas julgadas irrelevantes segundo alguns critérios:
*  Identificadores de pleito: ano_eleicao, codigo_eleicao, data_eleicao, descricao_eleicao, tipo_abrangencia_eleicao, codigo_tipo_eleicao, nome_tipo_eleicao
*  Identificadores de candidato: nome, nome_social, nome_urna, email, cpf, titulo_eleitoral, numero_processo_candidatura, numero_protocolo_candidatura, numero_sequencial, numero_urna e pergunta.
*  Dados não "numerizáveis": sigla_ue, descricao_ue, sigla_uf, sigla_uf_nascimento, codigo_municipio_nascimento, nome_municipio_nascimento, codigo_legenda, nome_legenda, sigla_legenda, idade_data_eleicao, data_nascimento, composicao_legenda, tipo_agremiacao
*  Descritores de variáveis ordenáveis: descricao_grau_instrucao, descricao_nacionalidade

Além disso, removeremos todos os candidatos cuja candidatura foi indeferida, bem como as colunas vinculadas a tal: codigo_detalhe_situacao_candidatura, codigo_situacao_candidatura, descricao_detalhe_situacao_candidatura e descricao_situacao_candidatura

Outro ponto foi a remoção de candidaturas a cargos não diretamente eletivos (Vice-presidente, Vice-governador e Vice-prefeito).

Também foram removidos 8 candidatos cujos registros de nascimento apresentam erros gritantes

As colunas booleanas concorre_reeleicao e declara_bens foram convertidas para inteiros (True = 1, False = 0)

Os partidos tiveram seus nomes e siglas atualizadas para a configuração atual

E finalmente, foi criada uma coluna "eleito", reunindo os dados de codigo_totalizacao_turno e descricao_totalizacao_turno, em uma configuração numérica (1 = eleito, 0 = não eleito). As colunas codigo_totalizacao_turno, descricao_totalizacao_turno e numero_turno foram a seguir descartadas

In [3]:
df_ordinarias_deferidas = tabela_candidato.loc[
    (tabela_candidato['codigo_tipo_eleicao'] == 2.0) &
    (tabela_candidato['codigo_situacao_candidatura'] == 12) &
    (~tabela_candidato['codigo_cargo'].isin([2, 4, 12])) &
    (tabela_candidato['idade_data_posse'].between(10, 200, inclusive=True)) &
    (tabela_candidato['codigo_totalizacao_turno'] != 6.0)
]
df_work = df_ordinarias_deferidas.drop([
    "ano_eleicao", 
    "codigo_eleicao", 
    "data_eleicao", 
    "descricao_eleicao", 
    "tipo_abrangencia_eleicao",
    "codigo_tipo_eleicao", 
    "nome_tipo_eleicao",
    "nome", 
    "nome_social", 
    "nome_urna",
    "numero_urna",
    "email",
    "cpf",
    "titulo_eleitoral",
    "numero_processo_candidatura", 
    "numero_protocolo_candidatura", 
    "numero_sequencial",
    "pergunta",
    "codigo_detalhe_situacao_candidatura", 
    "codigo_situacao_candidatura",
    "descricao_detalhe_situacao_candidatura", 
    "descricao_situacao_candidatura",
    "sigla_ue",
    "descricao_ue",
    "sigla_uf",
    "sigla_uf_nascimento",
    "codigo_municipio_nascimento",
    "nome_municipio_nascimento",
    "descricao_grau_instrucao",
    "descricao_nacionalidade",
    "codigo_legenda", 
    "nome_legenda", 
    "sigla_legenda",
    "composicao_legenda",
    "idade_data_eleicao",
    "data_nascimento",
    "tipo_agremiacao"
], axis=1)
df_work.rename(columns={
    'codigo_grau_instrucao':'grau_instrucao',
    'codigo_nacionalidade':'nacionalidade',
}, inplace=True)

df_work.loc[df_work['concorre_reeleicao'] == 'N', 'concorre_reeleicao'] = '0'
df_work.loc[df_work['concorre_reeleicao'] == 'S', 'concorre_reeleicao'] = '1'
df_work.loc[df_work['declara_bens'] == 'N', 'declara_bens'] = '0'
df_work.loc[df_work['declara_bens'] == 'S', 'declara_bens'] = '1'
df_work[["concorre_reeleicao", "declara_bens"]] = df_work[["concorre_reeleicao", "declara_bens"]].apply(pd.to_numeric)

df_work.loc[df_work['numero_partido'] == 15, 'nome_partido'] = 'MOVIMENTO DEMOCRATICO BRASILEIRO'
df_work.loc[df_work['numero_partido'] == 15, 'sigla_partido'] = 'MDB'
df_work.loc[df_work['numero_partido'] == 19, 'nome_partido'] = 'PODEMOS'
df_work.loc[df_work['numero_partido'] == 19, 'sigla_partido'] = 'PODE'
df_work.loc[df_work['numero_partido'] == 27, 'nome_partido'] = 'DEMOCRACIA CRISTA'
df_work.loc[df_work['numero_partido'] == 27, 'sigla_partido'] = 'DC'
df_work.loc[df_work['numero_partido'] == 51, 'nome_partido'] = 'PATRIOTA'
df_work.loc[df_work['numero_partido'] == 51, 'sigla_partido'] = 'PATRI'
df_work.loc[df_work['numero_partido'] == 70, 'nome_partido'] = 'AVANTE'
df_work.loc[df_work['numero_partido'] == 70, 'sigla_partido'] = 'AVANTE'
df_work.loc[df_work['numero_partido'] == 77, 'sigla_partido'] = 'SD'

df_work['eleito'] = np.where(df_work['codigo_totalizacao_turno'].between(1.0, 3.0, inclusive=True), 1, 0)
df_work = df_work.drop(["codigo_totalizacao_turno", "descricao_totalizacao_turno", "numero_turno"], axis=1)

df_work.head()

,codigo_cor_raca,codigo_estado_civil,codigo_genero,grau_instrucao,nacionalidade,codigo_ocupacao,descricao_cor_raca,descricao_estado_civil,descricao_genero,descricao_ocupacao,nome_partido,numero_partido,sigla_partido,codigo_cargo,concorre_reeleicao,declara_bens,descricao_cargo,despesa_maxima_campanha,idade_data_posse,eleito
1762732,2.0,3,2,6,1,999,PRETA,CASADO(A),MASCULINO,OUTROS,DEMOCRACIA CRISTA,27,DC,7,0,0,DEPUTADO ESTADUAL,400000.0,45.0,0
1762733,1.0,3,2,8,1,101,BRANCA,CASADO(A),MASCULINO,ENGENHEIRO,PARTIDO POPULAR SOCIALISTA,23,PPS,7,0,1,DEPUTADO ESTADUAL,1000000.0,50.0,0
1762734,3.0,1,2,8,1,111,PARDA,SOLTEIRO(A),MASCULINO,MEDICO,PARTIDO DOS TRABALHADORES,13,PT,7,0,1,DEPUTADO ESTADUAL,690000.0,48.0,0
1762735,3.0,1,2,6,1,298,PARDA,SOLTEIRO(A),MASCULINO,SERVIDOR PUBLICO MUNICIPAL,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,45,PSDB,7,0,1,DEPUTADO ESTADUAL,500000.0,39.0,0
1762736,3.0,3,2,8,1,115,PARDA,CASADO(A),MASCULINO,ODONTOLOGO,PARTIDO REPUBLICANO BRASILEIRO,10,PRB,7,0,0,DEPUTADO ESTADUAL,500000.0,40.0,0


O passo seguinte foi aplicar a técnica de One Hot Encoding, transformando as variáveis categóricas cor/raça, estado civil, gênero, ocupação, partido e cargo pretendido em múltiplos atributos booleanos

In [4]:
df_work = pd.concat([df_work,pd.get_dummies(df_work['descricao_cor_raca'], prefix='cor_raca')],axis=1)
df_work = pd.concat([df_work,pd.get_dummies(df_work['descricao_estado_civil'], prefix='estado_civil')],axis=1)
df_work = pd.concat([df_work,pd.get_dummies(df_work['descricao_genero'], prefix='genero')],axis=1)
df_work = pd.concat([df_work,pd.get_dummies(df_work['descricao_ocupacao'], prefix='ocupacao')],axis=1)
df_work = pd.concat([df_work,pd.get_dummies(df_work['sigla_partido'], prefix='partido')],axis=1)
df_work = pd.concat([df_work,pd.get_dummies(df_work['descricao_cargo'], prefix='cargo')],axis=1)
df_work.drop([
    'codigo_cor_raca', 'descricao_cor_raca',
    'codigo_genero', 'descricao_genero',
    'codigo_ocupacao', 'descricao_ocupacao', 
    'codigo_estado_civil','descricao_estado_civil',
    'nome_partido', 'numero_partido', 'sigla_partido',
    'codigo_cargo', 'descricao_cargo'
],axis=1, inplace=True)
df_work.head()

,grau_instrucao,nacionalidade,concorre_reeleicao,declara_bens,despesa_maxima_campanha,idade_data_posse,eleito,cor_raca_AMARELA,cor_raca_BRANCA,cor_raca_INDIGENA,...,cargo_1o SUPLENTE SENADOR,cargo_2o SUPLENTE SENADOR,cargo_DEPUTADO DISTRITAL,cargo_DEPUTADO ESTADUAL,cargo_DEPUTADO FEDERAL,cargo_GOVERNADOR,cargo_PREFEITO,cargo_PRESIDENTE,cargo_SENADOR,cargo_VEREADOR
1762732,6,1,0,0,400000.0,45.0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1762733,8,1,0,1,1000000.0,50.0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
1762734,8,1,0,1,690000.0,48.0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1762735,6,1,0,1,500000.0,39.0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1762736,8,1,0,0,500000.0,40.0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


## Principal Component Analysis

Normalizamos o dado e o separamos em treino e teste de modo a rodar o PCA separadamente em ambos

In [5]:
scaler = StandardScaler()
scaler.fit(df_work)
scaler.transform(df_work)
df_treino, df_teste = train_test_split(df_work, test_size=0.2)

c:\users\yami_\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype uint8, int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
c:\users\yami_\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint8, int32, int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
df_treino\
    .groupby(['eleito'])\
    .size()\
    .reset_index()\
    .rename(columns={0:'count'})

,eleito,count
0,0,347563
1,1,53640


In [8]:
df_teste\
    .groupby(['eleito'])\
    .size()\
    .reset_index()\
    .rename(columns={0:'count'})

,eleito,count
0,0,87122
1,1,13179


In [9]:
df_data = df_treino.drop(['eleito'], axis=1)
df_labels = df_treino.loc[:,['eleito']].values.ravel()
test_data = df_teste.drop(['eleito'], axis=1)
test_labels = df_teste.loc[:,['eleito']].values.ravel()

In [20]:
pca = decomposition.PCA(n_components=10)
pca_df_data = pca.fit_transform(df_data)
pca_test_data = pca.transform(test_data)

print(pca.explained_variance_ratio_[0])
print(pca.explained_variance_ratio_[1])
print(pca.explained_variance_ratio_[2])
print(pca.explained_variance_ratio_[3])
print(pca.explained_variance_ratio_[4])
print(pca.explained_variance_ratio_[5])
print(pca.explained_variance_ratio_[6])
print(pca.explained_variance_ratio_[7])
print(pca.explained_variance_ratio_[8])
print(pca.explained_variance_ratio_[9])

0.999999999900809
9.462111189299421e-11
2.226336828131404e-12
3.4536589894476016e-13
3.165619143852748e-13
2.5226956909229207e-13
1.4946719495534396e-13
1.0568257798924223e-13
9.526244407757431e-14
8.606454504548456e-14


## Multilayer Perceptron (Rede Neural)

In [21]:
mlpc = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
mlpc.fit(df_data_pca, df_labels)
mlpc_pred = mlpc.predict(pca_test_data)

In [22]:
mlpc_precision = precision_score(test_labels, mlpc_pred, average='macro')
mlpc_recall = recall_score(test_labels, mlpc_pred, average='macro')
mlpc_f1 = f1_score(test_labels, mlpc_pred, average='macro')
mlpc_cm = confusion_matrix(test_labels, mlpc_pred)

print("Precision is {}%".format(mlpc_precision * 100))
print("Recall is {}%".format(mlpc_recall * 100))
print("F-1 is {}%".format(mlpc_f1 * 100))
print(mlpc_cm)

Precision is 43.430274872633376%
Recall is 50.0%
F-1 is 46.48415616012976%
[[87122     0]
 [13179     0]]


c:\users\yami_\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\yami_\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Random Forest Classifier

In [23]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
rfc.fit(df_data_pca, df_labels)
rfc_pred = rfc.predict(pca_test_data)

In [24]:
rfc_precision = precision_score(test_labels, rfc_pred, average='macro')
rfc_recall = recall_score(test_labels, rfc_pred, average='macro')
rfc_f1 = f1_score(test_labels, rfc_pred, average='macro')
rfc_cm = confusion_matrix(test_labels, rfc_pred)

print("Precision is {}%".format(rfc_precision * 100))
print("Recall is {}%".format(rfc_recall * 100))
print("F-1 is {}%".format(rfc_f1 * 100))
print(rfc_cm)

Precision is 43.430274872633376%
Recall is 50.0%
F-1 is 46.48415616012976%
[[87122     0]
 [13179     0]]


c:\users\yami_\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\yami_\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Linear Regression

In [78]:
lr = LinearRegression()
lr.fit(df_data_pca, df_labels)
lr_pred = lr.predict(pca_test_data)

In [79]:
lr_precision = precision_score(test_labels, lr_pred, average='macro')
lr_recall = recall_score(test_labels, lr_pred, average='macro')
lr_f1 = f1_score(test_labels, lr_pred, average='macro') 
print("Precision is {}%".format(lr_precision * 100))
print("Recall is {}%".format(lr_recall * 100))
print("F-1 is {}%".format(lr_f1 * 100))

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [83]:
print(lr_pred)

[0.13344573 0.13341163 0.13254724 ... 0.13351547 0.13372603 0.13346338]


## SVM

In [ ]:
svmc = SVC(gamma='scale')
svmc.fit(df_data_pca, df_labels)
svmc_pred = svmc.predict(pca_test_data)

In [ ]:
svmc_precision = precision_score(test_labels, svmc_pred, average='macro')
svmc_recall = recall_score(test_labels, svmc_pred, average='macro')
svmc_f1 = f1_score(test_labels, svmc_pred, average='macro') 
print("Precision is {}%".format(svmc_precision * 100))
print("Recall is {}%".format(svmc_recall * 100))
print("F-1 is {}%".format(svmc_f1 * 100))